<a href="https://colab.research.google.com/github/Katinon/Self-Study/blob/main/%ED%98%BC%EC%9E%90-%EA%B3%B5%EB%B6%80%ED%95%98%EB%8A%94-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%94%A5%EB%9F%AC%EB%8B%9D/ValidationSet%2CGridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#교차 검증과 그리드 서치
#테스트세트를 사용하지 않고 과대-과소적합 측정방법
#훈련세트중 일부를 떼어 검증세트로 나눔
import pandas as pd
wine = pd.read_csv("http://bit.ly/wine_csv_data")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

#훈련세트를 다시 훈련세트와 검증세트로 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [4]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
# 모델 평가, 과대적합
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
# 교차 검증이란 검증 세트를 떼어 평가를 여러번 한 뒤, 점수를 평균화함
# 훈련세트를 세 부분으로 나누어 교차검증을 수행하는 것이 3-폴드-교차검증

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# 훈련하는 시간, 검증하는 시간, 검증 폴드의 점수

{'fit_time': array([0.00827432, 0.00717115, 0.00755119, 0.00725603, 0.00695348]), 'score_time': array([0.0008502 , 0.00073886, 0.00070643, 0.00070453, 0.00066519]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
import numpy as np
print(np.mean(scores['test_score']))
# 교차 검증의 최종 점수

0.855300214703487


In [7]:
# cross_validate()는 훈련 데이터를 섞지 않음
# 여기서는 train_test_split으로 섞었지만,
# 교차검증에서 훈련세트 섞으려면 분할기 지정해야함
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
# 만약 훈련세트를 섞고 10폴드 교차 검증을 수행하려면
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [9]:
# 최적의 매개변수를 찾는 그리드서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #모델과 파라미터와 사용할 cpu코어 개수 전달(전부 사용)
#cv 매개변수 기본값은 5, 5폴드 교차 검증 수행, 총 25번 계산
gs.fit(train_input, train_target)
#25 개 모델중에서 가장 높은 점수로 다시 훈련

dt = gs.best_estimator_ #일반 결정 트리처럼 사용할 수 있음
print(dt.score(train_input, train_target))
print(gs.best_params_) #최적의 파라미터
print(gs.cv_results_['mean_test_score']) # 각 매개변수에서 수행한 교차 검증의 평균 점수

best_index = np.argmax(gs.cv_results_['mean_test_score']) # 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [11]:
# 복잡한 매개변수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1), #정수
          'min_samples_split' : range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_estimator_)
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 점수

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=0.0004,
                       min_samples_split=12, random_state=42)
0.8683865773302731


In [14]:
# 매개변수 값을 미리 정하기 어려울 때, 그리드서치가 오래걸릴때 랜덤서치 이용
# 랜덤서치에는 매개변수를 샘플링하는 확률 분포 객체 전달

from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease' : uniform(0.0001, 0.001), #실수
          'max_depth' : randint(20, 50), #정수
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25),
          }

# n_iter 매개변수로 샘플링 횟수 지정
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [15]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
